# Web Scraping Notebook
## NBA Player Career Stats

In [1]:
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import pandas as pd
import time
import requests

In [2]:
f = open("players.txt", "r")
ply_list = f.read().split("\n")
print(ply_list)

['Jordan, Michael', 'Durant, Kevin', 'Iverson, Allen', 'Gervin, George', 'Malone, Karl', 'Bryant, Kobe', 'Wilkins, Dominique', 'Barry, Rick', 'Abdul-Jabbar, Kareem', 'Dantley, Adrian', 'Bird, Larry', 'Erving, Julius', 'Maravich, Peter', 'Thompson, David', 'Issel, Dan', 'King, Bernard', 'Barkley, Charles', 'McAdoo, Bob', 'Wade, Dwyane', 'Somerset, Willie', 'Olajuwon, Hakeem', 'English, Alex', 'Cunningham, Billy', 'Cousins, DeMarcus', 'Robinson, David', 'Hayes, Elvin', 'Ewing, Patrick', 'Richmond, Mitch', 'Nowitzki, Dirk', 'Webber, Chris', 'Drew, John', 'Scott, Charlie', 'Robinson III, Glenn', 'Arenas, Gilbert', 'Brisker, John', 'Drexler, Clyde', 'Malone, Moses', 'Free, World', 'Haywood, Spencer', 'McGinnis, George', 'Verga, Bob', 'Lanier, Bob', 'Johnson, Magic', 'Johnson, Marques', 'Aguirre, Mark', 'Mitchell, Mike', 'Pierce, Paul', 'Vandeweghe, Kiki', 'McGrady, Tracy', 'Thompson, Klay', 'Marbury, Stephon', 'Thomas, Isaiah', 'Thomas, Isiah', 'Bosh, Chris', 'Mashburn, Jamal', 'Jones, Larr

In [3]:
root = "https://www.basketball-reference.com/players/"
players = pd.DataFrame({"fname":[],
                        "lname":[],
                        "g":[],
                        "pts":[],
                        "trb":[],
                        "ast":[],
                        "fgp":[],
                        "fg3p":[],
                        "ftp":[],
                        "efgp":[],
                        "per":[],
                        "ws":[],
                         })
players.head()

,fname,lname,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws


In [4]:
# Ending pattern = 'last initial' + / + 'last[:5]' + 'first[:2]' + '01.html'
# Eg. j/jordami01.html

def rooter(item):
    sub = item.split(",")
    add = str(sub[0][0]) + "/" + str(sub[0][:5]) +  str(sub[1][:3]) + "01.html"
    add = add.replace(" ","").lower()
    return(str(root + add))
print(rooter('Stojakovic, Peja'))

https://www.basketball-reference.com/players/s/stojape01.html


In [5]:
def plyr_scrape(name, y):
    url = rooter(name)
    name = name.split(", ")
    fname = name[1]
    lname = name[0]
    headers = requests.utils.default_headers()
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    page = soup.prettify()
    page1 = soup.findAll("div", {"class": "p1"})
    page2 = soup.findAll("div", {"class": "p2"})
    page3 = soup.findAll("div", {"class": "p3"})
    x = 1
    page1 = str(page1).split("</div>")
    page2 = str(page2).split("</div>")
    page3 = str(page3).split("</div>")
    for n in page1:
        if "Games" in n:
            n = str(n).split("</h4><p></p>")
            g = n[1]
            g = str(g).replace("<p>","")
            g = str(g).replace("</p>","")
            g = str(g).replace("\n","")
        elif "Points" in n:
            n = str(n).split("</h4><p></p>")
            pts = n[1]
            pts = str(pts).replace("<p>","")
            pts = str(pts).replace("</p>","")
            pts = str(pts).replace("\n","")
        elif "Total Rebounds" in n:
            n = str(n).split("</h4><p></p>")
            trb = n[1]
            trb = str(trb).replace("<p>","")
            trb = str(trb).replace("</p>","")
            trb = str(trb).replace("\n","")
        elif "Assists" in n:
            n = str(n).split("</h4><p></p>")
            ast = n[1]
            ast = str(ast).replace("<p>","")
            ast = str(ast).replace("</p>","")
            ast = str(ast).replace("\n","")
    x = 1
    for n in str(page2).split("</h4><p></p>"):
        if x == 1:
            x += 1
            continue
        elif x == 2:            
            p = str(n).split("</p>',")
            fgp = p[0]
            fgp = str(fgp)[5:]
            x += 1
            continue
        elif x == 3:            
            p = str(n).split("</p>',")
            fg3p = p[0]
            fg3p = str(fg3p)[5:]
            x += 1
            continue
        elif x == 4:
            x += 1
            p = str(n).split("</p>',")
            ftp = p[0]
            ftp = str(ftp)[5:]
            continue
        elif x == 5:
            x += 1
            p = str(n).split("</p>',")
            efgp = p[0]
            efgp = str(efgp)[5:]
            continue
    x = 1
    for n in str(page3).split("</h4><p></p>"):
        if x == 1:
            x += 1
            continue
        elif x == 2:
            x += 1
            p = str(n).split("</p>',")
            per = p[0]
            per = str(per)[5:]
        elif x == 3:
            ws = n.split("</p>")
            ws = str(ws[0])[5:]
            x += 1
            continue
    df = pd.DataFrame({"fname":[fname],
                        "lname":[lname],
                        "g":[g],
                        "pts":[pts],
                        "trb":[trb],
                        "ast":[ast],
                        "fgp":[fgp],
                        "fg3p":[fg3p],
                        "ftp":[ftp],
                        "efgp":[efgp],
                        "per":[per],
                        "ws":[ws],
                         },index = [y])
    return df
    
print(plyr_scrape(ply_list[0], 1))

     fname   lname     g   pts  trb  ast   fgp  fg3p   ftp  efgp   per     ws
1  Michael  Jordan  1072  30.1  6.2  5.3  49.7  32.7  83.5  50.9  27.9  214.0


In [ ]:
y = 1
for player in ply_list:
    try:
        insert = plyr_scrape(player, y)
    except:
        continue
    players = players.append(insert)
    y += 1
players.head()

In [ ]:
players.head(400)

In [ ]:
players.to_csv("scrape.csv")

In [ ]:
def rookie(name, y):
    url = rooter(name)
    name = name.split(", ")
    fname = name[1]
    lname = name[0]
    headers = requests.utils.default_headers()
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    page = soup.prettify()
    height = soup.findAll("span",{"itemprop":"height"})
    weight = soup.findAll("span",{"itemprop":"weight"})
    birth = soup.findAll("span", {"itemprop":"birthDate"})
    b_place = soup.findAll("span", {"itemprop":"birthPlace"})
    href = soup.findAll("a")
    p = soup.findAll("p")
    
    height = str(height).split('"height">')
    height = str(height[1]).replace("</span>]","")
    
    weight = str(weight).split('"weight">')
    weight = str(weight[1]).replace("</span>]","")
    weight = weight[:5]
    
    b_place = str(b_place).split('"birthPlace">')
    b_place = b_place[1]
    b_place = str(b_place).split(">")
    
    city = str(b_place[0]).split(",")
    city = str(city[0]).replace("in", "")
    city = city.strip()
    
    state = b_place[1]
    state = str(state).split("<")[0]
    
    for i in href:
        if "/friv/colleges.fcgi?" in str(i):
            college = i
            college = str(college).split(">")
            college = str(college[1]).replace("</a","")
        elif "/boxscores/" in str(i) and '"/boxscores/"' not in str(i):
            debut = i
            debut = str(debut).split(">")
            debut = str(debut[1]).replace("</a","")
            debut = debut[-4:]
    
    df = pd.DataFrame({"fname":[fname],
                        "lname":[lname],
                        "school":[college],
                        "debut":[debut],
                        "ht":[height],
                        "wt":[weight],
                        "b_city":[city],
                        "b_state":[state],
                         },index = [y])
    return df
    
print(rookie(ply_list[0], 1))


In [ ]:
meta = pd.DataFrame({"fname":[],
                      "lname":[],
                      "school":[],
                      "debut":[],
                      "ht":[],
                      "wt":[],
                      "b_city":[],
                      "b_state":[],
                       })
meta.head()

In [ ]:
y = 1
for player in ply_list:
    try:
        insert = rookie(player, y)
    except:
        player = str(player).split(",")
        df = pd.DataFrame({"fname":[player[1]],
                        "lname":[player[0]],
                        "school":[""],
                        "debut":[""],
                        "ht":[""],
                        "wt":[""],
                        "b_city":[""],
                        "b_state":[""],
                         },index=[y])
        meta = meta.append(df)
        y += 1
        continue
    meta = meta.append(insert)
    y += 1
meta.head()

In [ ]:
scrape = pd.read_csv("master.csv")
scrape["debut"] = pd.to_datetime(scrape["debut"], format = "%Y").dt.striftime("%Y")
scrape.head(20)

In [ ]:
ult = pd.merge(scrape, meta,  how='outer', on=['fname','lname'])
ult.head(20)

In [ ]:
ult.to_csv("scrape2.csv")

In [ ]:
f = open("players2.txt", "r")
list2 = f.read().split("\n")
print(list2)

In [ ]:
meta2 = pd.DataFrame({"fname":[],
                      "lname":[],
                      "debut":[],
                      "ht":[],
                      "wt":[],
                      "b_city":[],
                      "b_state":[],
                       })
meta2.head()

In [6]:
def rookie2(name, y):
    url = rooter(name)
    name = name.split(", ")
    fname = name[1]
    lname = name[0]
    headers = requests.utils.default_headers()
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    page = soup.prettify()
    height = soup.findAll("span",{"itemprop":"height"})
    weight = soup.findAll("span",{"itemprop":"weight"})
    birth = soup.findAll("span", {"itemprop":"birthDate"})
    b_place = soup.findAll("span", {"itemprop":"birthPlace"})
    href = soup.findAll("a")
    p = soup.findAll("p")
    
    height = str(height).split('"height">')
    height = str(height[1]).replace("</span>]","")
    
    weight = str(weight).split('"weight">')
    weight = str(weight[1]).replace("</span>]","")
    weight = weight[:5]
    
    b_place = str(b_place).split('"birthPlace">')
    b_place = b_place[1]
    b_place = str(b_place).split(">")
    
    city = str(b_place[0]).split(",")
    city = str(city[0]).replace("in", "")
    city = city.strip()
    
    state = b_place[1]
    state = str(state).split("<")[0]
    
    for i in href:
        if "/boxscores/" in str(i) and '"/boxscores/"' not in str(i):
            debut = i
            debut = str(debut).split(">")
            debut = str(debut[1]).replace("</a","")
            debut = debut[-4:]
    
    df = pd.DataFrame({"fname":[fname],
                        "lname":[lname],
                        "debut":[debut],
                        "ht":[height],
                        "wt":[weight],
                        "b_city":[city],
                        "b_state":[state],
                         },index = [y])
    return df
    
print(rookie2("Bryant, Kobe", 1))


  fname   lname debut   ht     wt        b_city       b_state
1  Kobe  Bryant  1996  6-6  212lb  Philadelphia  Pennsylvania


In [ ]:
y = 1
for player in list2:
    try:
        insert = rookie2(player, y)
    except:
        continue
    meta2 = meta2.append(insert)
    y += 1
meta2.head()

In [ ]:
scrape2 = pd.read_csv("scrape2.csv")
scrape2.head(20)

In [ ]:
ult2 = pd.merge(scrape2, meta2,  how='outer', on=['fname','lname'])
ult.head(20)

In [ ]:
ult2.to_csv("scrape.csv")

In [7]:
df = pd.read_csv("master.csv")
df.head(20)

,Unnamed: 0,fname_x,lname,school,debut,wt,ht(inches),b_city,b_state,g,...,ast,fgp,fg3p,ftp,efgp,per,ws,fname_y,lat,long
0,0,Paul,Silas,Creighton,1964.0,220lb,79.0,Prescott,Arkansas,1254.0,...,2.1,43.2,-,67.3,43.2,13.9,79.9,5.0,NaN,NaN
1,1,Kevin,McKenna,Creighton,1981.0,195lb,77.0,St. Paul,Minnesota,243.0,...,1.4,41.4,36.7,81.9,45.6,10.5,4.0,5.0,NaN,NaN
2,2,Benoit,Benjamin,Creighton,1985.0,250lb,NaN,Monroe,Louisiana,807.0,...,1.3,49.7,4.8,72.1,49.7,14.8,32.7,5.0,NaN,NaN
3,3,Chad,Gallagher,Creighton,1994.0,255lb,82.0,Rockford,Illinois,2.0,...,0.0,100,-,-,100,66.8,0.1,5.0,NaN,NaN
4,4,Rodney,Buford,Creighton,1999.0,189lb,77.0,Milwaukee,Wisconsin,230.0,...,0.8,41.2,30,75.8,43.6,10,2.9,5.0,NaN,NaN
5,5,David,Andersen,Augsburg College,1967.0,230lb,82.0,Mneapolis,Minnesota,140.0,...,1.1,48.1,-,72.7,48.1,17.5,10.9,3.0,NaN,NaN
6,6,Daniel,Anderson,Augsburg College,1967.0,230lb,82.0,Mneapolis,Minnesota,140.0,...,1.1,48.1,-,72.7,48.1,17.5,10.9,3.0,NaN,NaN
7,7,Devean,George,Augsburg College,1999.0,220lb,80.0,Mneapolis,Minnesota,630.0,...,0.9,39.2,34.3,72.1,45.6,10.6,18.2,3.0,NaN,NaN
8,8,Jim,Caldwell,Georgia Tech,1967.0,240lb,82.0,Durham,North Carolina,137.0,...,2.0,42.5,13.3,63.1,42.6,13.2,5.7,30.0,33.7756,84.3963
9,9,Brook,Steppe,Georgia Tech,1982.0,195lb,77.0,Chapel Hill,North Carolina,238.0,...,1.2,46.9,31,82,47.4,12.5,4.6,30.0,33.7756,84.3963


In [12]:
yr = df(["fname","lname","ht(inches)"])
yr.groupby(["debut"]).mean()
yr.head(20)

TypeError: 'DataFrame' object is not callable

In [ ]:
yr.to_csv("y_avg.csv")

In [ ]:
school = df.groupby(["school"]).count()
school.head(20)

In [ ]:
school.to_csv("school.csv")

In [37]:
mstr = pd.read_csv("master.csv")
temp = pd.read_csv("school.csv")
ult2 = pd.merge(mstr, temp,  how='outer', on="school")
ult2.head(20)

,fname_x,lname,school,debut,wt,ht(inches),b_city,b_state,g,pts,...,ftp,efgp,per,ws,fname_y,lat_x,long_x,fname,lat_y,long_y
0,Alaa,Abdelnaby,Duke,1990.0,240lb,82.0,Cairo,Egypt,256.0,5.7,...,70.1,50.2,13,4.8,38.0,36.0014,78.9382,38.0,36.0014,78.9382
1,Mark,Alarie,Duke,1986.0,217lb,80.0,Phoenix,Arizona,325.0,7.5,...,77.5,48.3,13,8.4,38.0,36.0014,78.9382,38.0,36.0014,78.9382
2,William,Avery,Duke,1999.0,197lb,74.0,Augusta,Georgia,142.0,2.7,...,71.4,37.8,7.3,-0.9,38.0,36.0014,78.9382,38.0,36.0014,78.9382
3,Gene,Banks,Duke,1981.0,215lb,79.0,Philadelphia,Pennsylvania,468.0,11.3,...,73,53.9,15.1,31.2,38.0,36.0014,78.9382,38.0,36.0014,78.9382
4,Shane,Battier,Duke,2001.0,220lb,80.0,Birmgham,Michigan,977.0,8.6,...,74.3,52.9,12.6,75.7,38.0,36.0014,78.9382,38.0,36.0014,78.9382
5,Carlos,Boozer,Duke,2002.0,258lb,81.0,Aschaffenburg,Germany,861.0,16.2,...,72.2,52.1,19.5,80.3,38.0,36.0014,78.9382,38.0,36.0014,78.9382
6,Elton,Brand,Duke,1999.0,275lb,80.0,Cortlandt Manor,New York,1058.0,15.9,...,73.6,50.1,20.5,109.6,38.0,36.0014,78.9382,38.0,36.0014,78.9382
7,Andre,Dawkins,Duke,2014.0,215lb,77.0,Fairfax,Virginia,4.0,0.8,...,-,25,-5,-0.1,38.0,36.0014,78.9382,38.0,36.0014,78.9382
8,Johnny,Dawkins,Duke,1986.0,165lb,74.0,Washgton,District of Columbia,541.0,11.1,...,85.7,47.4,14.7,21.8,38.0,36.0014,78.9382,38.0,36.0014,78.9382
9,Luol,Deng,Duke,2004.0,237lb,81.0,Wau,South Sudan,902.0,14.8,...,76.9,48.5,15.4,74.0,38.0,36.0014,78.9382,38.0,36.0014,78.9382


In [38]:
mod = ult2.groupby(["b_state"]).mean()
mod.head(50)


,debut,ht(inches),g,pts,trb,ast,ws,fname_y,lat_x,long_x,fname,lat_y,long_y
b_state,,,,,,,,,,,,,
Alabama,1988.015873,78.903226,334.857143,7.239683,3.593651,1.266667,18.887302,17.147541,35.202878,88.451441,17.147541,35.202878,88.451441
Alaska,2008.000000,74.000000,646.000000,8.900000,2.500000,3.700000,32.300000,48.000000,38.954300,95.255800,48.000000,38.954300,95.255800
Argentina,2007.416667,78.750000,311.583333,5.366667,2.600000,1.350000,18.783333,12.500000,39.980000,75.160000,12.500000,39.980000,75.160000
Arizona,1995.200000,76.700000,266.700000,6.010000,2.050000,1.470000,10.470000,21.800000,34.958362,100.704837,21.800000,34.958362,100.704837
Arkansas,1988.878788,78.161290,420.636364,6.824242,3.212121,1.390909,23.060606,15.625000,37.704421,94.330005,15.625000,37.704421,94.330005
Australia,2005.250000,81.000000,138.750000,3.525000,2.666667,0.675000,5.950000,21.500000,37.385071,93.443857,21.500000,37.385071,93.443857
Bahamas,1987.000000,80.333333,329.666667,8.233333,3.533333,1.133333,19.833333,19.666667,37.071100,91.630433,19.666667,37.071100,91.630433
Belgium,2005.500000,76.000000,719.500000,10.600000,2.300000,3.100000,56.050000,48.000000,38.954300,95.255800,48.000000,38.954300,95.255800
Bosnia and Herzegovina,2004.750000,82.222222,185.222222,5.766667,2.433333,0.844444,6.355556,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
mod.to_csv("state_avg.csv")

#mod = ult2.groupby(["b_state"]).mean()
#mod.head(50)